<a href="https://colab.research.google.com/github/Sarztak/WindyGraph-A-graph-based-recommender-for-restaurants-in-Chicago/blob/main/Factorization_Model_Machine_Learning_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving processed_restaurant_data.pkl to processed_restaurant_data (2).pkl


In [ ]:
from google.colab import files

uploaded = files.upload()


Saving processed_review_data.pkl to processed_review_data (3).pkl


In [ ]:
!pip install lightfm

In [ ]:
!pip install numpy==1.24.4 scikit-surprise


In [ ]:
!pip install numpy

In [ ]:
import pandas as pd

review_restaurantdata = pd.read_pickle("processed_review_data.pkl")
restaurant_data = pd.read_pickle("processed_restaurant_data.pkl")


In [ ]:
# Merge on restaurant_id (review_restaurantdata) and id (restaurant_data)
merge_reviewndata = review_restaurantdata.merge(restaurant_data[['id', 'categories_list']], left_on='restaurant_id', right_on='id')

# Prepare interaction dataset: user_id, restaurant_id, rating, categories_list
merge_reviewndata = merge_reviewndata[['user_id', 'restaurant_id', 'rating', 'categories_list']].dropna()

merge_reviewndata['categories_list'] = merge_reviewndata['categories_list']

# Faster training
merge_reviewndata = merge_reviewndata.sample(n=5000, random_state=123)


In [ ]:
# Set up LightFM dataset
from lightfm.data import Dataset
dataset = Dataset()

# Collect all features first
all_features = set()

for _, row in restaurant_data.iterrows():
    features = row['categories_list'] + [
        f"pop_score:{row['popularity_score']:.2f}",
        f"rating:{row['normalized_rating_restaurants']:.2f}",
        f"log_reviews:{row['normalized_log_review_count']:.2f}",
        f"loc_lat:{row['normalized_latitude']:.2f}",
        f"loc_lon:{row['normalized_longitude']:.2f}"
    ]
    all_features.update(features)

# Re-fit dataset with all features
dataset.fit(
    users=merge_reviewndata['user_id'].unique(),           # review data user IDs
    items=restaurant_data['id'].unique(),                  # include all restaurant IDs
    item_features=all_features
)


In [ ]:
# Train the LightFM model
from lightfm import LightFM
model = LightFM(loss='warp', no_components=30)
model.fit(interactions, item_features=item_features, epochs=10, num_threads=4)


In [ ]:
import pickle
import joblib

with open("lightfm_model.pkl", "wb") as f:
    pickle.dump(model, f)

joblib.dump(dataset, "lightfm_dataset.joblib")
joblib.dump(interactions, "interactions_matrix.joblib")
joblib.dump(item_features, "item_features_matrix.joblib")

print("files saved.")


files saved.


In [ ]:
import pickle
import joblib
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset

# Load saved model and components
with open("lightfm_model.pkl", "rb") as f:
    model = pickle.load(f)

dataset = joblib.load("lightfm_dataset.joblib")
interactions = joblib.load("interactions_matrix.joblib")
item_features = joblib.load("item_features_matrix.joblib")

user_id_map = dataset.mapping()[0]
item_id_map = dataset.mapping()[2]

# Inverse maps
id_to_user = {v: k for k, v in user_id_map.items()}
id_to_item = {v: k for k, v in item_id_map.items()}

def recommend_known_user(user_id_str, model, dataset, interactions, item_features, N=5):
    user_id_map = dataset.mapping()[0]
    item_id_map = dataset.mapping()[2]
    item_index_to_id = {v: k for k, v in item_id_map.items()}

    try:
        user_idx = user_id_map[user_id_str]
    except KeyError:
        print(f"User ID '{user_id_str}' not found.")
        return []

    n_items = interactions.shape[1]
    scores = model.predict(user_ids=user_idx,
                           item_ids=np.arange(n_items),
                           item_features=item_features)

    known_items = interactions.tocsr()[user_idx].indices
    scores[known_items] = -np.inf

    top_items = np.argsort(-scores)[:N]
    return [item_index_to_id[i] for i in top_items]


In [ ]:
merge_reviewndata['user_id'].unique()[:10]


array(['thV5RSlc8l4QL6Jf4--PIw', 'VqjxVozyfl8-BVNZ5g3D8Q',
       'RiB1mlLFirfWDqjTv09eAA', 'tH_RQBcxqYAmB-A4RVLHaw',
       'Sw-IGWHmnXji2z43x4A5OA', 'Y_NVgxBRcII68dSuwfNHAQ',
       'ukBY6FqUM0NCf7C1w8am1w', 'wrGhtevrzlMRcbGo72Aubg',
       '3xUa12mLDyWAb3ht5iVW1w', '37RrURW8yCJoo1DEgdiiYg'], dtype=object)

In [ ]:
list(dataset.mapping()[0].keys())[:10]


['thV5RSlc8l4QL6Jf4--PIw',
 'VqjxVozyfl8-BVNZ5g3D8Q',
 'RiB1mlLFirfWDqjTv09eAA',
 'tH_RQBcxqYAmB-A4RVLHaw',
 'Sw-IGWHmnXji2z43x4A5OA',
 'Y_NVgxBRcII68dSuwfNHAQ',
 'ukBY6FqUM0NCf7C1w8am1w',
 'wrGhtevrzlMRcbGo72Aubg',
 '3xUa12mLDyWAb3ht5iVW1w',
 '37RrURW8yCJoo1DEgdiiYg']

In [ ]:
user = "RiB1mlLFirfWDqjTv09eAA"

# Generate recommendations
recommended_items = recommend_known_user(
    user, model, dataset, interactions, item_features, N=5
)

# Print recommendations with names
print(f"Top recommendations for user '{user}':")
for item_id in recommended_items:
    name = id_to_name.get(item_id, "[Unknown]")
    print(f"- {name} (ID: {item_id})")



Top recommendations for user 'RiB1mlLFirfWDqjTv09eAA':
- Vincent Restaurant (ID: XVDQYUFhJ4Ur_92eVYs3CA)
- Jermie Food Mart (ID: 1uFx_OTe0pFvBRuyADkF3Q)
- Jerk & Soul Food (ID: gQkjAD_UzAyf4qCwKYB4zQ)
- Buffalo Wild Wings (ID: GQO4bNj7UlQS14bZISAImw)
- Babee's Spiritual Supply (ID: P4c-GUAlDRjAaHTzQ7DLmw)


In [ ]:
from lightfm.evaluation import precision_at_k

precision = precision_at_k(model, interactions, k=5, item_features=item_features).mean()
print(f"Precision@5: {precision:.4f}")



Precision@5: 0.0458


In [ ]:
from lightfm.evaluation import recall_at_k

recall = recall_at_k(model, interactions, k=5, item_features=item_features).mean()
print(f"Recall@5: {recall:.4f}")


Recall@5: 0.2046


In [ ]:
from lightfm.evaluation import auc_score

auc = auc_score(model, interactions, item_features=item_features).mean()
print(f"AUC: {auc:.4f}")


AUC: 0.9837


In [ ]:
from lightfm.cross_validation import random_train_test_split

train_interactions, test_interactions = random_train_test_split(interactions, test_percentage=0.3, random_state=123)


In [ ]:
# Step 1: Re-fit Dataset with Feature Names to ensure full coverage
dataset.fit(
    users=merge_reviewndata['user_id'],
    items=restaurant_data['id'],
    item_features=set(
        sum(restaurant_data['categories_list'], []) +
        [f"pop_score:{v:.2f}" for v in restaurant_data['popularity_score']] +
        [f"rating:{v:.2f}" for v in restaurant_data['normalized_rating_restaurants']] +
        [f"log_reviews:{v:.2f}" for v in restaurant_data['normalized_log_review_count']] +
        [f"loc_lat:{v:.2f}" for v in restaurant_data['normalized_latitude']] +
        [f"loc_lon:{v:.2f}" for v in restaurant_data['normalized_longitude']]
    )
)

# Step 2: Build item_features ONCE
item_features = dataset.build_item_features([
    (row['id'],
     row['categories_list'] + [
         f"pop_score:{row['popularity_score']:.2f}",
         f"rating:{row['normalized_rating_restaurants']:.2f}",
         f"log_reviews:{row['normalized_log_review_count']:.2f}",
         f"loc_lat:{row['normalized_latitude']:.2f}",
         f"loc_lon:{row['normalized_longitude']:.2f}"
     ])
    for _, row in restaurant_data.iterrows()
])

# Step 3: Train-test split
from lightfm.cross_validation import random_train_test_split
train_interactions, test_interactions = random_train_test_split(
    interactions, test_percentage=0.2, random_state=123
)

# Step 4: Train model with tuned hyperparameters (boost latent factors and epochs)
from lightfm import LightFM
model = LightFM(loss='warp', no_components=100, learning_rate=10)
model.fit(train_interactions, item_features=item_features, epochs=40, num_threads=4)

# Step 5: Evaluate recall@10
from lightfm.evaluation import recall_at_k
recall = recall_at_k(model, test_interactions, item_features=item_features, k=10).mean()
print(f"Recall@10: {recall:.4f}")


Recall@10: 0.0032


In [ ]:
# Refit with binarized ratings
merge_reviewndata['implicit_rating'] = (merge_reviewndata['rating'] >= 4).astype(int)
(interactions, _) = dataset.build_interactions([
    (row['user_id'], row['restaurant_id'], row['implicit_rating'])
    for _, row in merge_reviewndata.iterrows()
])

# Model
model = LightFM(loss='bpr', no_components=150, learning_rate=0.05)
model.fit(train_interactions, item_features=item_features, epochs=60, num_threads=4)

# Evaluate
recall = recall_at_k(model, test_interactions, item_features=item_features, k=20).mean()
print(f"Recall@20: {recall:.4f}")


Recall@20: 0.0011
